In [1]:
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
import re
import time
import collections
import os
import itertools
from tqdm import tqdm

In [2]:
def build_dataset(words, n_words, atleast=1):
    count = [['GO', 0], ['PAD', 1], ['EOS', 2], ['UNK', 3]]
    counter = collections.Counter(words).most_common(n_words)
    counter = [i for i in counter if i[1] >= atleast]
    count.extend(counter)
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

def add_start_end(string):
    string = string.split()
    strings = []
    for s in string:
        if s[-1] == '-':
            s = s[:-1]
        s = list(s)
        s[0] = '<%s'%(s[0])
        s[-1] = '%s>'%(s[-1])
        strings.extend(s)
    return strings

In [3]:
with open('stemmer-data-v5.txt','r') as fopen:
    texts = fopen.readlines()

In [4]:
after, before = [], []
for i in texts:
    splitted = i.encode('ascii', 'ignore').decode("utf-8").lower().split('\t')
    if len(splitted) < 2:
        continue
    after.append(add_start_end(splitted[1]))
    before.append(add_start_end(splitted[0]))
    
len(after), len(before)

(41528, 41528)

In [5]:
before[:5]

[['<s', 'p', 'i', 'd', 'e', 'r>'],
 ['<t', 'r', 'a', 'n', 's', 'e', 'k', 's', 'u', 'a', 'l>'],
 ['<k', 'h', 'u', 's', 'u', 's>'],
 ['<k', 'r', 'i', 'm', 'i', 'n', 'o', 'l', 'o', 'g', 'i>'],
 ['<n', 'e', 'g', 'e', 'r', 'i', 'p', 'i', 'n', 'g', 'a', 't>']]

In [6]:
after[:5]

[['<s', 'p', 'i', 'd', 'e', 'r>'],
 ['<t', 'r', 'a', 'n', 's', 'e', 'k', 's', 'u', 'a', 'l>'],
 ['<k', 'h', 'u', 's', 'u', 's>'],
 ['<k', 'r', 'i', 'm', 'i', 'n', 'o', 'l', 'o', 'g', 'i>'],
 ['<n', 'e', 'g', 'e', 'r', 'i', 'p', 'i', 'n', 'g', 'a', 't>']]

In [7]:
concat_from = list(itertools.chain(*before))
vocabulary_size_from = len(list(set(concat_from)))
data_from, count_from, dictionary_from, rev_dictionary_from = build_dataset(concat_from, vocabulary_size_from)
print('vocab from size: %d'%(vocabulary_size_from))
print('Most common words', count_from[4:10])
print('Sample data', data_from[:10], [rev_dictionary_from[i] for i in data_from[:10]])
print('filtered vocab size:',len(dictionary_from))
print("% of vocab used: {}%".format(round(len(dictionary_from)/vocabulary_size_from,4)*100))

vocab from size: 80
Most common words [('a', 44151), ('e', 26905), ('n', 22703), ('i', 21583), ('r', 16110), ('u', 12472)]
Sample data [27, 20, 7, 19, 5, 38, 35, 8, 4, 6] ['<s', 'p', 'i', 'd', 'e', 'r>', '<t', 'r', 'a', 'n']
filtered vocab size: 84
% of vocab used: 105.0%


In [8]:
concat_to = list(itertools.chain(*after))
vocabulary_size_to = len(list(set(concat_to)))
data_to, count_to, dictionary_to, rev_dictionary_to = build_dataset(concat_to, vocabulary_size_to)
print('vocab from size: %d'%(vocabulary_size_to))
print('Most common words', count_to[4:10])
print('Sample data', data_to[:10], [rev_dictionary_to[i] for i in data_to[:10]])
print('filtered vocab size:',len(dictionary_to))
print("% of vocab used: {}%".format(round(len(dictionary_to)/vocabulary_size_to,4)*100))

vocab from size: 80
Most common words [('a', 35149), ('e', 18010), ('i', 17934), ('n', 15750), ('r', 12021), ('u', 11371)]
Sample data [20, 21, 6, 17, 5, 32, 30, 8, 4, 7] ['<s', 'p', 'i', 'd', 'e', 'r>', '<t', 'r', 'a', 'n']
filtered vocab size: 84
% of vocab used: 105.0%


In [9]:
GO = dictionary_from['GO']
PAD = dictionary_from['PAD']
EOS = dictionary_from['EOS']
UNK = dictionary_from['UNK']

In [10]:
for i in range(len(after)):
    after[i].append('EOS')

In [11]:
class Stemmer:
    def __init__(self, size_layer, num_layers, embedded_size, 
                 from_dict_size, to_dict_size, learning_rate, 
                 dropout = 0.8, beam_width = 15, force_teaching_ratio=0.5):
        
        def lstm_cell(reuse=False):
            return tf.nn.rnn_cell.LSTMCell(size_layer, reuse=reuse)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype=tf.int32)
        batch_size = tf.shape(self.X)[0]

        encoder_embeddings = tf.Variable(tf.random_uniform([from_dict_size, embedded_size], -1, 1))
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        encoder_cells = tf.nn.rnn_cell.MultiRNNCell([lstm_cell() for _ in range(num_layers)])
        self.encoder_out, self.encoder_state = tf.nn.dynamic_rnn(cell = encoder_cells, 
                                                                 inputs = encoder_embedded, 
                                                                 sequence_length = self.X_seq_len,
                                                                 dtype = tf.float32)
        
        encoder_state = tuple(self.encoder_state[-1] for _ in range(num_layers))
        main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        decoder_embeddings = tf.Variable(tf.random_uniform([to_dict_size, embedded_size], -1, 1))
        dense_layer = tf.layers.Dense(to_dict_size)

        with tf.variable_scope('decode'):
            attention_mechanism = tf.contrib.seq2seq.LuongAttention(
            num_units = size_layer, 
            memory = encoder_embedded,
            memory_sequence_length = self.X_seq_len)
            decoder_cell = tf.contrib.seq2seq.AttentionWrapper(
                cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell() for _ in range(num_layers)]),
                attention_mechanism = attention_mechanism,
                attention_layer_size = size_layer)
            main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
            decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
            training_helper = tf.contrib.seq2seq.ScheduledEmbeddingTrainingHelper(
            inputs = tf.nn.embedding_lookup(decoder_embeddings, decoder_input),
                sequence_length = self.Y_seq_len,
                embedding = decoder_embeddings,
                sampling_probability = 1 - force_teaching_ratio,
                time_major = False)
            training_decoder = tf.contrib.seq2seq.BasicDecoder(
                cell = decoder_cell,
                helper = training_helper,
                initial_state = decoder_cell.zero_state(batch_size, tf.float32).clone(cell_state=encoder_state),
                output_layer = dense_layer)
            training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = training_decoder,
                impute_finished = True,
                maximum_iterations = tf.reduce_max(self.Y_seq_len))

        with tf.variable_scope('decode', reuse=True):
            encoder_out_tiled = tf.contrib.seq2seq.tile_batch(encoder_embedded, beam_width)
            encoder_state_tiled = tf.contrib.seq2seq.tile_batch(encoder_state, beam_width)
            X_seq_len_tiled = tf.contrib.seq2seq.tile_batch(self.X_seq_len, beam_width)
            attention_mechanism = tf.contrib.seq2seq.LuongAttention(
                num_units = size_layer, 
                memory = encoder_out_tiled,
                memory_sequence_length = X_seq_len_tiled)
            decoder_cell = tf.contrib.seq2seq.AttentionWrapper(
                cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell(reuse=True) for _ in range(num_layers)]),
                attention_mechanism = attention_mechanism,
                attention_layer_size = size_layer)
            predicting_decoder = tf.contrib.seq2seq.BeamSearchDecoder(
                cell = decoder_cell,
                embedding = decoder_embeddings,
                start_tokens = tf.tile(tf.constant([GO], dtype=tf.int32), [batch_size]),
                end_token = EOS,
                initial_state = decoder_cell.zero_state(batch_size * beam_width, tf.float32).clone(cell_state = encoder_state_tiled),
                beam_width = beam_width,
                output_layer = dense_layer,
                length_penalty_weight = 0.0)
            predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = predicting_decoder,
                impute_finished = False,
                maximum_iterations = 2 * tf.reduce_max(self.X_seq_len))
            
            
        self.training_logits = training_decoder_output.rnn_output
        self.predicting_ids = tf.identity(predicting_decoder_output.predicted_ids[:, :, 0],name="logits")
        
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
                                                     targets = self.Y,
                                                     weights = masks)
        
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [12]:
size_layer = 128
num_layers = 2
embedded_size = 64
learning_rate = 1e-3
batch_size = 128
epoch = 10

In [13]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Stemmer(size_layer, num_layers, embedded_size, len(dictionary_from), 
                len(dictionary_to), learning_rate)
sess.run(tf.global_variables_initializer())

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [14]:
def str_idx(corpus, dic, UNK=3):
    X = []
    for i in corpus:
        ints = []
        for k in i:
            ints.append(dic.get(k, UNK))
        X.append(ints)
    return X

In [15]:
X = str_idx(before, dictionary_from)
Y = str_idx(after, dictionary_to)

In [16]:
from sklearn.cross_validation import train_test_split
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.2)

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [17]:
def pad_sentence_batch(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(len(sentence))
    return padded_seqs, seq_lens

In [18]:
from tqdm import tqdm
from sklearn.utils import shuffle
import time

for EPOCH in range(epoch):
    lasttime = time.time()
    total_loss, total_accuracy, total_loss_test, total_accuracy_test = 0, 0, 0, 0
    train_X, train_Y = shuffle(train_X, train_Y)
    test_X, test_Y = shuffle(test_X, test_Y)
    pbar = tqdm(range(0, len(train_X), batch_size), desc='train minibatch loop')
    for k in pbar:
        index = min(k+batch_size,len(train_X))
        batch_x, seq_x = pad_sentence_batch(train_X[k: k+batch_size], PAD)
        batch_y, seq_y = pad_sentence_batch(train_Y[k: k+batch_size], PAD)
        acc, loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                                      feed_dict={model.X:batch_x,
                                                model.Y:batch_y})
        total_loss += loss
        total_accuracy += acc
        pbar.set_postfix(cost=loss, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_X), batch_size), desc='test minibatch loop')
    for k in pbar:
        index = min(k+batch_size,len(test_X))
        batch_x, seq_x = pad_sentence_batch(test_X[k: k+batch_size], PAD)
        batch_y, seq_y = pad_sentence_batch(test_Y[k: k+batch_size], PAD)
        acc, loss = sess.run([model.accuracy, model.cost], 
                                      feed_dict={model.X:batch_x,
                                                model.Y:batch_y})
        total_loss_test += loss
        total_accuracy_test += acc
        pbar.set_postfix(cost=loss, accuracy = acc)
        
    total_loss /= (len(train_X) / batch_size)
    total_accuracy /= (len(train_X) / batch_size)
    total_loss_test /= (len(test_X) / batch_size)
    total_accuracy_test /= (len(test_X) / batch_size)
        
    print('epoch: %d, avg loss: %f, avg accuracy: %f'%(EPOCH, total_loss, total_accuracy))
    print('epoch: %d, avg loss test: %f, avg accuracy test: %f'%(EPOCH, total_loss_test, total_accuracy_test))

train minibatch loop:   1%|          | 2/260 [00:00<00:14, 17.23it/s, accuracy=0.758, cost=0.891]

epoch: 0, avg loss: 1.332408, avg accuracy: 0.670189
epoch: 0, avg loss test: 0.987139, avg accuracy test: 0.735963


train minibatch loop:   1%|          | 2/260 [00:00<00:15, 16.79it/s, accuracy=0.761, cost=0.809]

epoch: 1, avg loss: 0.864946, avg accuracy: 0.760973
epoch: 1, avg loss test: 0.741874, avg accuracy test: 0.787838


train minibatch loop:   1%|          | 2/260 [00:00<00:19, 13.38it/s, accuracy=0.856, cost=0.475]

...

epoch: 7, avg loss: 0.219649, avg accuracy: 0.934410
epoch: 7, avg loss test: 0.231226, avg accuracy test: 0.931023


train minibatch loop:   1%|          | 2/260 [00:00<00:15, 16.72it/s, accuracy=0.945, cost=0.176]

epoch: 8, avg loss: 0.189146, avg accuracy: 0.943443
epoch: 8, avg loss test: 0.196329, avg accuracy test: 0.941194


test minibatch loop: 100%|██████████| 65/65 [00:02<00:00, 31.72it/s, accuracy=0.947, cost=0.174]

epoch: 9, avg loss: 0.168858, avg accuracy: 0.949601
epoch: 9, avg loss test: 0.175603, avg accuracy test: 0.947418


In [19]:
predicted = sess.run(model.predicting_ids, 
                     feed_dict={model.X:batch_x})

In [20]:
for i in range(len(batch_x)):
    print('row %d'%(i+1))
    print('BEFORE:',''.join([rev_dictionary_from[n] for n in batch_x[i] if n not in [0,1,2,3]]))
    print('REAL AFTER:',''.join([rev_dictionary_to[n] for n in batch_y[i] if n not in[0,1,2,3]]))
    print('PREDICTED AFTER:',''.join([rev_dictionary_to[n] for n in predicted[i] if n not in[0,1,2,3]]),'\n')

row 1
BEFORE: <selcat>
REAL AFTER: <selcat>
PREDICTED AFTER: <selcat> 

row 2
BEFORE: <enphc>
REAL AFTER: <enphc>
PREDICTED AFTER: <enphc> 

row 3
BEFORE: <activists>
REAL AFTER: <activists>
PREDICTED AFTER: <activists> 

row 4
BEFORE: <kelembapan>
REAL AFTER: <lembap>
PREDICTED AFTER: <lembap> 

row 5
BEFORE: <dok>
REAL AFTER: <dok>
PREDICTED AFTER: <dok> 

row 6
BEFORE: <skiers>
REAL AFTER: <skiers>
PREDICTED AFTER: <skiers> 

row 7
BEFORE: <gfmas>
REAL AFTER: <gfmas>
PREDICTED AFTER: <gfmas> 

row 8
BEFORE: <shamsul>
REAL AFTER: <shamsul>
PREDICTED AFTER: <shamsul> 

row 9
BEFORE: <menakutkan>
REAL AFTER: <takut>
PREDICTED AFTER: <akut> 

row 10
BEFORE: <transcoding>
REAL AFTER: <transcoding>
PREDICTED AFTER: <transcoding> 

row 11
BEFORE: <sejat>
REAL AFTER: <sejat>
PREDICTED AFTER: <sejat> 

row 12
BEFORE: <exceptional>
REAL AFTER: <exceptional>
PREDICTED AFTER: <exceptional> 

row 13
BEFORE: <atu>
REAL AFTER: <atu>
PREDICTED AFTER: <atu> 

row 14
BEFORE: <ssdhaliwal>
REAL AFTER: 